In [1]:
from utilities import *
from astropy.io import fits
from astropy.table import QTable,vstack
import glob
import radio_beam
import sys

source='DSi'
fnum=fields[source]

linemodelhome='/blue/adamginsburg/d.jeff/SgrB2DS_COMAnalysis/linemodels/'
linemodelversion='firstrelease'
linemodelpath=linemodelhome+f'{linemodelversion}/{source}/'

targetmolecule='CH3OCHO'
targetmoleculepath=linemodelpath+f'{targetmolecule}.fits'
targetmoltable=QTable.read(targetmoleculepath)

othermoleculepaths=glob.glob(linemodelpath+'*.fits')
othermoleculepaths.remove(targetmoleculepath)

ch3ohpath=ch3oh_sourcedict[source]
targetpix=pixdict[source]
origsourcepath=f'/blue/adamginsburg/d.jeff/SgrB2DSreorg/field{fnum}/CH3OH/{source}{ch3ohpath}'
contpath=origsourcepath+'reprojectedcontinuum.fits'

fwhmpath=glob.glob(origsourcepath+'*fwhm*')[0]
fwhmmap=fits.getdata(fwhmpath)*u.km/u.s
fwhm_at_pix=fwhmmap[targetpix[0],targetpix[1]]

reprojcontfits=fits.open(contpath)
reprojcont=reprojcontfits[0].data*u.Jy
reprojcontrestfreq=225*u.GHz#manual addition 11/9/2022, wiggle room w/i GHz
cntmbeam=radio_beam.Beam.from_fits_header(reprojcontfits[0].header)
reprojcont_K=reprojcont.to('K',cntmbeam.jtok_equiv(reprojcontrestfreq))
continuumlevel=reprojcont_K[targetpix[0],targetpix[1]]

In [2]:
safelines=[]
minimumcontaminantthreshold=0.1#If contaminant lines have more than this times the target line's flux, the target line is considered contaminated and removed
firstline=True
for targetline in targetmoltable:
    flux=targetline['ModelBrightness']
    reffreq=targetline['ReferenceFrequency']
    qn=targetline['QNs']
    if flux <= continuumlevel:
        continue
    else:
        possiblecontaminants={}
        for tablepath in othermoleculepaths:
            othermolecule=tablepath.replace(f'{linemodelpath}','')
            othermolecule=othermolecule.replace('.fits','')
            
            othermoltable=QTable.read(tablepath)
            fwhm_Hz=velocitytofreq(fwhm_at_pix,reffreq)
            diff=np.abs(othermoltable['ReferenceFrequency']-reffreq)
            findcontam=np.where(diff<=fwhm_Hz)[0]
            if len(findcontam) > 0:
                candidate_contaminant_lines=othermoltable[findcontam]
                numcontaminants=len(candidate_contaminant_lines)
                if numcontaminants > 1:
                    contaminantflux=np.sum(candidate_contaminant_lines['ModelBrightness'])
                elif numcontaminants == 1:
                    contaminantflux=candidate_contaminant_lines['ModelBrightness']
                if contaminantflux >= (flux*minimumcontaminantthreshold):
                    print(f'{targetmolecule}\'s {qn} @ {reffreq} likely contaminated by {numcontaminants} {othermolecule} lines')
                    possiblecontaminants.update({othermolecule:candidate_contaminant_lines})
        if len(possiblecontaminants) == 0:
            if firstline:
                safelines=QTable(targetline)
                firstline=False
            else:
                safelines.add_row(targetline)

CH3OCHO's 19.416-18.415 @ 233.1692869279879 GHz likely contaminated by 1 C2H5OH lines


In [7]:
#safelines=QTable()
safelinepathbase=linemodelpath+'safelines/'
for targetline in safelines:
    doublets=QTable()
    flux=targetline['ModelBrightness']
    reffreq=targetline['ReferenceFrequency']
    qn=targetline['QNs']
    fwhm_Hz=velocitytofreq(fwhm_at_pix,reffreq)
    diff=np.abs(safelines['ReferenceFrequency']-reffreq)
    findcontam=np.where(diff<=fwhm_Hz)[0]
    print(f'{targetline['QNs']}: {findcontam}')
    if len(findcontam) > 1:
        print(f'{qn} has {len(findcontam)-1} companion lines')
        if len(doublets) == 0:
            doublets=vstack([doublets,QTable(safelines[findcontam])])
        elif qn not in doublets['QNs']:
            doublets=vstack([doublets,QTable(safelines[findcontam])])
    #safelines=vstack([safelines,QTable(targetline)])

    doubletdirectory=safelinepathbase+f'doublets/{targetmolecule}/'
    if os.path.exists(doubletdirectory) == False and len(doublets) > 0:
        os.mkdir(doubletdirectory)
        doubletspath=doubletdirectory+f'/{qn}.fits'
        doublets.write(doubletspath)
    elif len(doublets) > 0:
        doubletspath=doubletdirectory+f'/{qn}.fits'
        doublets.write(doubletspath,overwrite=True)
if os.path.exists(doubletdirectory) == False:
    print(f'{targetmolecule} has no doublets')

1816-1715: [0]
18.216-17.215: [1]
2020-1919: [2 3 4 5]
2020-1919 has 3 companion lines
20.120-19.119: [2 3 4 5]
20.120-19.119 has 3 companion lines
20.020-19.019: [2 3 4 5]
20.020-19.019 has 3 companion lines
2020-1919: [2 3 4 5]
2020-1919 has 3 companion lines
17.314-16.313: [6 7]
17.314-16.313 has 1 companion lines
17.314-16.313: [6 7]
17.314-16.313 has 1 companion lines
17.314-16.313: [8]
19.416-18.415: [9]
18.414-17.413: [10]
18.414-17.413: [11]


In [4]:
safelines

QNs,RestFrequency,ReferenceFrequency,Eupper,Degeneracy,Aij,ModelBrightness,ModelNupper
,GHz,GHz,K,,Hz,K,1 / cm2
bytes13,float64,float64,float64,int64,float64,float64,float64
1816-1715,216.830197,216.78976570869528,105.67841609660555,74,0.0001479677211578287,16.825091198307746,66689853319460.086
18.216-17.215,216.838889,216.7984560879396,105.66789854254446,74,0.0001480024364726828,16.8282876388437,66692685472496.74
2020-1919,216.96476500000003,216.92430861644306,111.4989717040179,82,0.00015313544122337337,18.744347822559856,72183058508145.53
20.120-19.119,216.9659004,216.9254438047304,111.48118536134272,82,0.00015316241265626233,18.748622734872594,72188242574001.55
20.020-19.019,216.9662462,216.92578954025072,111.49889891271856,82,0.00015313643126118543,18.74422397012688,72183079723378.03
2020-1919,216.96742,216.92696312137787,111.4811144129527,82,0.00015319870168211061,18.752641506068112,72188263253597.84
17.314-16.313,218.2809,218.2401982030351,99.72992293662472,70,0.000150776835327309,16.40718147939958,64618525222914.57
17.314-16.313,218.2809,218.2401982030351,99.72992293662472,70,0.000150776835327309,16.40718147939958,64618525222914.57


In [5]:
doublets

<QTable length=0>
<No columns>

In [4]:
fwhm_Hz

<Quantity 3.14128644 MHz>

In [13]:
safelinepath=safelinepathbase+f'{targetmolecule}.fits'
if not os.path.exists(safelinepath):
    os.mkdir(safelinepathbase)
safelines.write(safelinepath)#,overwrite=True)

In [14]:
safelines

QNs,RestFrequency,ReferenceFrequency,Eupper,Degeneracy,Aij,ModelBrightness,ModelNupper
,GHz,GHz,K,,Hz,K,1 / cm2
bytes24,float64,float64,float64,int64,float64,float64,float64
"13(0,13)(1)-12(1,12)(1)",217.26230230000002,217.22808692869543,135.54154448503328,27,6.272140276018215e-05,12.714210465445003,106946490947998.53
"21(5,16)(2)-21(4,17)(2)",218.5545074,218.52008852685998,226.01385361044416,43,7.18477867162649e-05,15.047079787784515,112459870588564.12
"13(2,11)(0)-12(2,10)(0)",230.6725581,230.63623082581753,138.6199773674354,27,0.00010602668351003007,18.52904542476179,105446581329916.16
"16(5,11)(2)-16(4,12)(2)",230.9537832,230.91741163731874,145.77872777262775,33,7.765111093374898e-05,16.1091008872317,124714879149827.9
"14(0,14)(2)-13(1,13)(2)",230.9913834,230.9550059158828,85.52671483307333,29,0.00011962150933731016,27.88287735035063,144497225766071.06
"21(5,17)(2)-21(4,18)(2)",231.5585678,231.52210099332368,225.95122187481488,43,8.357364920231331e-05,15.57784941265251,112492191730495.92
"20(5,16)(2)-20(4,17)(2)",231.5609215,231.52445432265318,208.21222307878986,41,8.265677678007207e-05,15.921879461215342,116354777484665.55
"14(1,14)(0)-13(1,13)(0)",231.6687419,231.63225774264802,141.90345196927348,29,0.00011080795735279709,20.22470284564705,111564016120728.23


In [8]:
((230.9913834-230.955)/230.955)*c.to('km s-1')

<Quantity 47.22768036 km / s>

In [5]:
0.00016236727257136008*c.to('km s-1')

<Quantity 48.67648374 km / s>